In [1]:
from selenium import webdriver
from core import MySQLcompatible
from bs4 import BeautifulSoup
import re
import utils
import json
import time

insert into ignore

In [2]:
tabelas = {}

tabelas['jogos_uni'] = (
"CREATE TABLE `jogos_uni` ("
"  `id` int(11) NOT NULL,"
"  `titulo` varchar(250) NOT NULL,"
"  `data` datetime NOT NULL,"
"  `slugLiga` varchar(120) DEFAULT NULL,"
"  `pais` varchar(100) DEFAULT NULL,"
"  `liga` varchar(100) DEFAULT NULL,"
"  `status` int(11) NOT NULL,"
"  `posicao` int(11) NOT NULL"
") ENGINE=MyISAM DEFAULT CHARSET=latin1;")

tabelas['modal_uni'] = (
"CREATE TABLE IF NOT EXISTS `modal_uni` ("
"  `id` int(11) NOT NULL,"
"  `jogo_id` int(11) NOT NULL,"
"  `odd_id` int(11) NOT NULL,"
"  `cat_id` int(11) NOT NULL,"
"  `categoria` varchar(250) NOT NULL,"
"  `propriedade` varchar(250) NOT NULL,"
"  `valor` decimal(8,2) NOT NULL DEFAULT '0.00',"
"  `status` int(11) NOT NULL"
") ENGINE=MyISAM DEFAULT CHARSET=latin1;")

In [3]:
options = webdriver.ChromeOptions()
options.add_argument('headless')
browser = webdriver.Chrome(options=options)
browser.get("https://bets93.net/")
browser.find_element_by_xpath("//div[@class='lateral']/div/ul").click() #clica no botao "todos"

In [4]:
pattern_campeonato = re.compile(r"c_visivel")
pattern_jogo = re.compile(r"j_visivel_")

In [5]:
soup = BeautifulSoup(browser.page_source, "html.parser")

In [39]:
tabela_jogos = soup.find(class_="jogos")
jogos = tabela_jogos.findAll("div",recursive=False)
# c_visivel = tabela_jogos.findAll(id="c_visivel")
# j_visivel = tabela_jogos.findAll(id=pattern_jogo)

In [ ]:
# for jogo in c_visivel:
#     print(jogo.find(class_="camp").text)

In [ ]:
# for jogo in j_visivel:
#     print(jogo.attrs)

In [101]:
for jogo in jogos:
    attr = jogo.get("id")
    if pattern_campeonato.match(attr):
        NOME_CAMP = jogo.find(class_="camp").text
        slugLiga = NOME_CAMP.lower().replace(' ','-')
        slugLiga = re.sub(pattern=r'(^-|-$)',repl='', string=slugLiga)
        pais = NOME_CAMP.split()[0]
        liga = NOME_CAMP.split()[1:]
        liga = ' '.join(liga)
        status = 1
        posicao = 1
    elif pattern_jogo.match(attr):
        id_jogo = attr.split('_')[-1]
        titulo = jogo.find(class_="times fundojogos").text.split()[:-3]
        titulo = ' '.join(titulo)
        data_hora = jogo.find(class_="datahora").text
        dados_jogo = {
        'id_jogo':id_jogo, 'titulo': titulo, 'data_hora': data_hora, 'slugLiga': slugLiga,
        'pais':pais, 'liga': liga, 'status': status, 'posicao': posicao}
        print(json.dumps(dados_jogo, indent=4))

{
    "id_jogo": "82916039",
    "titulo": "Hertha Berlin x Fortuna Dusseldorf",
    "data_hora": " 04/10/2019 - 14:30 ",
    "slugLiga": "alemanha-bundesliga",
    "pais": "Alemanha",
    "liga": "Bundesliga",
    "status": 1,
    "posicao": 1
}
{
    "id_jogo": "82916041",
    "titulo": "Bayern Munich x TSG Hoffenheim",
    "data_hora": " 05/10/2019 - 09:30 ",
    "slugLiga": "alemanha-bundesliga",
    "pais": "Alemanha",
    "liga": "Bundesliga",
    "status": 1,
    "posicao": 1
}
{
    "id_jogo": "82916043",
    "titulo": "Bayer Leverkusen x RB Leipzig",
    "data_hora": " 05/10/2019 - 09:30 ",
    "slugLiga": "alemanha-bundesliga",
    "pais": "Alemanha",
    "liga": "Bundesliga",
    "status": 1,
    "posicao": 1
}
{
    "id_jogo": "82916045",
    "titulo": "SC Freiburg x Borussia Dortmund",
    "data_hora": " 05/10/2019 - 09:30 ",
    "slugLiga": "alemanha-bundesliga",
    "pais": "Alemanha",
    "liga": "Bundesliga",
    "status": 1,
    "posicao": 1
}
{
    "id_jogo": "829160